In [1]:
import importlib
import numpy
import sqlite3

try:
    importlib.reload(wi_parser)
except NameError:
    import wi_parser

In [ ]:
# test loading a particular entry from the CIR file
rxID = 25
cirData25 = wi_parser.loadEntryCIR("data/simple_walk/lora multifreq rosslyn.cir.t002_05.r009.p2m", rxID)
print("{} paths found for rxID {}".format(len(cirData25), rxID))

# test loading all entries from CIR file
cirData = wi_parser.loadAllCIR("data/simple_walk/lora multifreq rosslyn.cir.t002_05.r009.p2m", 9)
# get rid of entries without CIR data
cirData = {rxid:cir for rxid,cir in cirData.items() if len(cir) > 0}
print("{} RX entries found".format(len(cirData)))

In [10]:
# project and data parameters
project="lora multifreq rosslyn"
studyarea="simple walk"
projectdir="data"
txset = 3
txID = 2
rxset = 9

In [3]:
# get transmitter and reciever positions from SQL file
sqlfile = wi_parser.sqlfilename(project=project,
                         studyarea=studyarea,
                         projectdir=projectdir)
print("using SQL file:", sqlfile)

txpos = wi_parser.parsePos(sqlfile, devType="txset", setid=txset)
print("# of TX entries:", len(txpos))
rxpos = wi_parser.parsePos(sqlfile, devType="rxset", setid=rxset)
print("# of RX entries:", len(rxpos))

# find line of sight distances between sets of transmitters and receivers
distances = wi_parser.computeDistancesLOS(txpos, rxpos)
print("# of distance entries:", len(distances))

# find ideal time of arrival between tx and rx devices
toa = wi_parser.computeToaLOS(distances)
print("# of ToA entries:", len(toa))

using SQL file: data/simple walk/lora multifreq rosslyn.simple walk.sqlite
# of TX entries: 5
# of RX entries: 1849
# of distance entries: 9245
# of ToA entries: 9245


In [11]:
# parse CIR entries
cirfile = wi_parser.outputfilename(project=project,
                                   studyarea=studyarea,
                                   output="cir",
                                   txset=txset,
                                   txn=txID, 
                                   rxset=rxset, 
                                   projectdir=projectdir)
print("using output file:", cirfile)
cir = wi_parser.loadAllCIR(cirfile, rxset, txset, txID)
# remove any entries which have zero paths
cir = {linkid:cirentry for linkid,cirentry in cir.items() if len(cirentry) > 0}
print("# of (non empty) CIR entries:", len(cir))

using output file: data/simple walk/lora multifreq rosslyn.cir.t002_03.r009.p2m
# of (non empty) CIR entries: 701


In [12]:
# double check to make sure we're indexing things correctly
print("common keys:", len(cir.keys() & toa.keys()))

common keys: 701


In [17]:
for i, key in enumerate(cir):
#     print(key)
    if i > 20:
        break

print("key:", key)
print("cir:", *cir[key], sep='\n')

key: ((3, 2), (9, 343))
cir:
(1.0158e-06, (-1.3398742693201454e-13-6.273008049135774e-14j))
(1.03684e-06, (5.1397705158917125e-14-1.786096459165768e-14j))
(9.77003e-07, (-1.3575625987955785e-14+1.3846042853596742e-14j))
(1.01726e-06, (1.7458199212276029e-16-1.8983733820752155e-14j))
(1.03858e-06, (9.721597549028112e-15+3.4927472133035916e-15j))
(9.83786e-07, (3.1454616682085937e-15-1.537373479128762e-15j))
(9.79107e-07, (-1.8937075975035083e-15+2.857069667430739e-15j))
(9.85896e-07, (4.897886157676064e-16-3.789956389212342e-16j))
